 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
source("https://bioconductor.org/biocLite.R")

Bioconductor version 3.7 (BiocInstaller 1.30.0), ?biocLite for help


In [4]:
biocLite()

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.1 (2018-07-02).
Old packages: 'git2r'


In [5]:
biocLite("scran")

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.7 (BiocInstaller 1.30.0), R 3.5.1 (2018-07-02).
Installing package(s) ‘scran’



The downloaded binary packages are in
	/var/folders/1w/m236pxg93fqfspflhhqj5g9h0000gn/T//Rtmpoz6WSO/downloaded_packages


Old packages: 'git2r'


In [ ]:
dataA <- read.table("~/Downloads/DataA3_2.txt")

In [103]:
dataF <- read.table("~/Downloads/DataF3_2.txt")

In [1]:
library(scran)

ERROR: Error in library(scran): there is no package called ‘scran’


In [105]:
dir.create("results", showWarning=FALSE)

In [106]:
logDataA3 <- as.matrix(dataA)
logDataF3 <- as.matrix(dataF)

In [107]:
raw.all <- cbind(logDataF3, logDataA3)

In [108]:
first.batch <- rep(c(TRUE, FALSE), c(ncol(logDataF3), ncol(logDataA3)))

In [109]:
# Making a plotting function.
plotFUN <- function(fname, Y, subset=NULL, ..., xlab="tSNE 1",ylab="tSNE 2",main="") {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col=ifelse(first.batch, "black", "black")[subset],
       bg="white", xlab=xlab, ylab=ylab, main=main) 
    dev.off()
}

In [110]:
batchcolor=c("black","yellow")
plotFUNb <- function(fname, Y, subset=NULL, ...) {
  if (is.null(subset)) {
    subset <- seq_len(nrow(Y))
  }
  png(fname,width=900,height=700)
  par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
  plot(Y[,1], Y[,2], cex=2,
       pch=ifelse(first.batch, 21, 1)[subset], 
       col="black",
       bg="white")#,  xlab="tSNE 1",ylab="tSNE 2")
  dev.off()
    }

In [111]:
X.unc <- raw.all

In [112]:
t.unc <- t(X.unc)

In [113]:
require(Rtsne)


In [114]:
set.seed(0)

In [115]:
all.dists.unc <- as.matrix(dist(t.unc))

In [116]:
tsne.unc <- Rtsne(all.dists.unc, is_distance=TRUE, perplexity = 90)

In [117]:
plotFUN("results/uncFA.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

In [118]:
plotFUNb("results/uncFAb.png", tsne.unc$Y, main="Uncorrected",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

In [119]:
rm(all.dists.unc)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3814967,203.8,5684620,303.6,5684620,303.6
Vcells,19402029,148.1,54004798,412.1,54003441,412.1


In [120]:
mnn.out<-mnnCorrect(logDataF3, logDataA3,k=20, sigma=0.1,cos.norm.in=TRUE, cos.norm.out=TRUE, var.adj=TRUE,compute.angle=TRUE)

X.mnn <- cbind(mnn.out$corrected[[1]], mnn.out$corrected[[2]])
t.mnn <- t(X.mnn)

In [121]:
dim(X.mnn)
dim(unique(X.mnn))

[1] 1000 2000

[1]  505 2000

In [122]:
# plot histogram of angles between batch vectors and 2 svds of the reference batch.
png(file="results/angles.png",width=900,height=700)
par(mfrow=c(1,1),mar=c(6,6,4,2),cex.axis=2,cex.main=3,cex.lab=2.5)
hist(mnn.out$angles[[2]],xlab="Angle",ylab="Frequency",main="")
dev.off()

pdf 
  2

In [123]:
# Generating a t-SNE plot.
set.seed(0)
all.dists.mnn <- as.matrix(dist(t.mnn))
tsne.mnn <- Rtsne(all.dists.mnn, is_distance=TRUE, perplexity = 90)
plotFUN("results/mnnFA.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")
plotFUNb("results/mnnFAb3.png", tsne.mnn$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

pdf 
  2

In [125]:
set.seed(0)
tsne.mnn2 <- Rtsne(t.mnn, perplexity = 90, check_duplicates = FALSE)
plotFUN("results/mnnFA_conventsne.png", tsne.mnn2$Y, main="MNN",  xlab="tSNE 1",ylab="tSNE 2")

pdf 
  2

In [124]:
head(dataA)

,AAACATACCAAGCT.1,AAACATACCCCTAC.1,AAACATACCCGTAA.1,AAACATACCCTCGT.1,AAACATACGAGGTG.1,AAACATACGCGAAG.1,AAACATACGTCGTA.1,AAACATACGTTGCA.1,AAACATACTATGGC.1,AAACATACTCAGGT.1,⋯,ACACATCTCGCCTT.11,ACACATCTCGTTAG.1,ACACATCTCTGGAT.1,ACACCAGAAAGGCG.1,ACACCAGAAATCGC.1,ACACCAGAATGCTG.1,ACACCAGACCTATT.1,ACACCAGAGACGGA.1,ACACCAGAGATGAA.1,ACACCAGAGCGTTA.1
MIR1302-10,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
RP11-34P13.8,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
